In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install torch torchvision transformers pillow

In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Load the pre-trained CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

# SUNRGBD Embeddings

In [ ]:
import torch
from PIL import Image
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm

# Configuration
model_name = "openai/clip-vit-base-patch32"
project_dir = "/content/drive/MyDrive/NNDL/NNDL Project"
csv_path = f"{project_dir}/image_depth_pairs.csv"
output_csv_path = f"{project_dir}/clip_embeddings_sunrgbd.csv"
batch_size = 32

# Load model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

# Load CSV
df = pd.read_csv(csv_path)

# Ensure 'depth' column exists
if 'depth' not in df.columns:
    df['depth'] = df['image'].apply(lambda x: x.rsplit('.', 1)[0] + ".png")

# Split list into batches
def batchify(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Function to load images with fallback
def load_images(paths):
    images = []
    for path in paths:
        try:
            images.append(Image.open(path).convert("RGB"))
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            images.append(Image.new('RGB', (224, 224), (0, 0, 0)))
    return images

# Function to compute embeddings
def compute_image_embeddings(paths):
    embeddings = []
    for batch_paths in tqdm(list(batchify(paths, batch_size)), desc="Processing images"):
        images = load_images(batch_paths)
        image_inputs = processor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**image_inputs)
            image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        embeddings.extend(image_features.cpu().numpy().tolist())
    return embeddings

# Compute embeddings
image_embeddings_list = compute_image_embeddings(df['image'].tolist())
depthmap_embeddings_list = compute_image_embeddings(df['depth'].tolist())

# # Compute caption embeddings
# caption_embeddings_list = []
# for batch_texts in tqdm(list(batchify(df['caption'].tolist(), batch_size)), desc="Processing captions"):
#     text_inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
#     with torch.no_grad():
#         text_features = model.get_text_features(**text_inputs)
#         text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)
#     caption_embeddings_list.extend(text_features.cpu().numpy().tolist())

# Save results
output_df = pd.DataFrame({
    "image": df['image'],
    "depth": df['depth'],
    # "caption": df['caption'],
    "image_embedding": image_embeddings_list,
    "depth_embedding": depthmap_embeddings_list,
    # "caption_embedding": caption_embeddings_list
})

output_df.to_csv(output_csv_path, index=False)
print(f"Embeddings saved to {output_csv_path}")

Processing images: 100%|██████████| 33/33 [07:18<00:00, 13.29s/it]


Embeddings saved to /content/drive/MyDrive/NNDL/NNDL Project/clip_embeddings_sunrgbd.csv


### Test Image Embedding

In [ ]:
## Test image embedding

# Load an image
image_path = "/content/drive/MyDrive/NNDL/NNDL Project/SUNRGBDv2Test/11082015/2015-11-08T13.42.16.610-0000006334/image/2015-11-08T13.42.16.610-0000006334.jpg"
image = Image.open(image_path).convert("RGB")

# Preprocess the image
inputs = processor(images=image, return_tensors="pt").to(device)

# Generate image embeddings
with torch.no_grad():
    image_features = model.get_image_features(**inputs)

# Normalize the embeddings
image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

print("Image embedding shape:", image_features.shape)
print("Embedding vector:", image_features)

Image embedding shape: torch.Size([1, 512])
Embedding vector: tensor([[-7.5607e-03,  3.0492e-02, -2.7465e-04,  3.3386e-02,  1.9184e-02,
         -1.9838e-02,  8.6550e-03,  2.2186e-02,  1.3725e-02,  1.5361e-02,
         -6.3274e-03, -8.7179e-03,  8.9925e-03, -2.1899e-02,  5.9433e-02,
         -1.2660e-02,  1.9098e-02, -2.8857e-02,  4.1614e-02,  7.3074e-03,
          2.4522e-02,  1.3411e-02, -1.0740e-03, -1.4817e-02,  1.1826e-03,
          5.1101e-02,  2.9038e-02, -5.0248e-02,  9.9804e-03,  4.4920e-03,
         -1.4693e-02,  4.1576e-02, -1.3138e-02, -2.5753e-02, -2.3634e-02,
          1.9156e-02, -1.4789e-02, -5.6320e-02, -2.5285e-02,  3.4392e-02,
         -2.6873e-03, -7.6433e-03,  2.7846e-02, -1.2606e-03, -4.7446e-03,
         -1.2161e-02, -4.0803e-03, -2.9121e-04, -1.7792e-02,  1.6905e-02,
          5.0490e-02,  1.6168e-03,  1.2044e-02,  1.8701e-02, -1.1980e-02,
         -1.1089e-02, -2.7840e-02, -3.9777e-02, -3.1672e-02,  2.2379e-02,
         -1.5061e-01, -5.1855e-02, -7.8255e-03, -5

# Image and Caption Embeddings

In [ ]:
import torch
from PIL import Image
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm

# Configuration
model_name = "openai/clip-vit-base-patch32"
project_dir = "/content/drive/MyDrive/NNDL/NNDL Project"
csv_path = f"{project_dir}/nyu2_train_captions.csv"
output_csv_path = f"{project_dir}/clip_embeddings.csv"
batch_size = 32

# Load model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

# Load CSV
df = pd.read_csv(csv_path)

# Split list into batches
def batchify(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Prepare data storage
image_embeddings_list = []
caption_embeddings_list = []

# Process images in batches
image_paths = df['image'].tolist()
captions = df['caption'].tolist()
for batch_paths in tqdm(list(batchify(image_paths, batch_size)), desc="Processing images"):
    images = []
    for path in batch_paths:
        try:
            images.append(Image.open(path).convert("RGB"))
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            images.append(Image.new('RGB', (224, 224), (0, 0, 0)))  # fallback blank image

    image_inputs = processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**image_inputs)
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

    image_embeddings_list.extend(image_features.cpu().numpy().tolist())

# Process text captions in batches
for batch_texts in tqdm(list(batchify(captions, batch_size)), desc="Processing captions"):
    text_inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
        text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)

    caption_embeddings_list.extend(text_features.cpu().numpy().tolist())

# Save results
output_df = pd.DataFrame({
    "image": df['image'],
    "caption": df['caption'],
    "image_embedding": image_embeddings_list,
    "caption_embedding": caption_embeddings_list
})

output_df.to_csv(output_csv_path, index=False)
print(f"Embeddings saved to {output_csv_path}")

Processing captions: 100%|██████████| 89/89 [05:43<00:00,  3.86s/it]


Embeddings saved to /content/drive/MyDrive/NNDL/NNDL Project/clip_embeddings.csv


# Image, Caption and Depth Map Embeddings

In [ ]:
import torch
from PIL import Image
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm

# Configuration
model_name = "openai/clip-vit-base-patch32"
project_dir = "/content/drive/MyDrive/NNDL Project"
csv_path = f"{project_dir}/sunrgbd_captions_for_trisha.csv"
output_csv_path = f"{project_dir}/clip_embeddings_for_trisha.csv"
batch_size = 32

# Load model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

# Load CSV
df = pd.read_csv(csv_path)
df = df[df['image'] != 'image']

# Ensure 'depthmap' column exists
# if 'depth' not in df.columns:
#     df['depthmap'] = df['image'].apply(lambda x: x.rsplit('.', 1)[0] + ".png")

# Split list into batches
def batchify(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Function to load images with fallback
def load_images(paths):
    images = []
    for path in paths:
        try:
            images.append(Image.open(path).convert("RGB"))
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            images.append(Image.new('RGB', (224, 224), (0, 0, 0)))
    return images

# Function to compute embeddings
def compute_image_embeddings(paths):
    embeddings = []
    for batch_paths in tqdm(list(batchify(paths, batch_size)), desc="Processing images"):
        images = load_images(batch_paths)
        image_inputs = processor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = model.get_image_features(**image_inputs)
            image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        embeddings.extend(image_features.cpu().numpy().tolist())
    return embeddings

# Compute embeddings
image_embeddings_list = compute_image_embeddings(df['image'].tolist())
depthmap_embeddings_list = compute_image_embeddings(df['depth'].tolist())

# Compute caption embeddings
caption_embeddings_list = []
for batch_texts in tqdm(list(batchify(df['caption'].tolist(), batch_size)), desc="Processing captions"):
    text_inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
        text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)
    caption_embeddings_list.extend(text_features.cpu().numpy().tolist())

# Save results
output_df = pd.DataFrame({
    "image": df['image'],
    "depthmap": df['depth'],
    "caption": df['caption'],
    "image_embedding": image_embeddings_list,
    "depthmap_embedding": depthmap_embeddings_list,
    "caption_embedding": caption_embeddings_list
})

output_df.to_csv(output_csv_path, index=False)
print(f"Embeddings saved to {output_csv_path}")

Processing captions: 100%|██████████| 44/44 [00:02<00:00, 17.09it/s]


Embeddings saved to /content/drive/MyDrive/NNDL Project/clip_embeddings_for_trisha.csv
